<a href="https://colab.research.google.com/github/kumaramardeep342/Colab-Work/blob/main/Tuning_%2B_Scratch_%2B_Adaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
- Dataset - CIFAR10 & FashionMNIST
- Pre-Trained Model - ResNet18
- Shallow Tunning
- Deep Tunning
- Training from Scratch
- Domain Adoption - how is model performance on different dataset.


# Import all the necessary libraries

In [2]:
# data anyalysis
import pandas as pd
import numpy as np
# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
#modeling using Pytorch
import torch
import torch.utils.data
from torchvision import transforms
from torchvision.datasets import CIFAR10, FashionMNIST
from torchvision.models import resnet18, ResNet18_Weights
from tqdm import tqdm
!pip install torchmetrics
from torchmetrics import Accuracy



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 12.0 MB/s eta 0:00:00


# Setup the Data Class (CIFAR10 & FashionMNIST)


In [3]:
# this is where we define the data
class DataModule():
  #save all the hyper - parameters here
    def __init__(self, batch_size,fmnist=False):
        super().__init__()
        self.batch_size = batch_size
        self.fmnist = fmnist

        # We define some augmentations that we would like to apply during training
        self.train_transform = transforms.Compose([
            transforms.Resize(256),
            transforms.RandomCrop(224, 4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

        # During validation we need to only normalize and resize
        self.val_transform = transforms.Compose([
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

        if self.fmnist:
          self.train_transform = transforms.Compose([
          transforms.transforms.Grayscale(3),
          self.train_transform
      ])
          self.val_transform = transforms.Compose([
          transforms.transforms.Grayscale(3),
          self.val_transform
      ])

    # This function sets up our datasets
    # which includes downloading and applying the augmentations
    def prepare_data(self):
      if self.fmnist:
        self.train_set = FashionMNIST(root='./data', train=True,download=True, transform=self.train_transform)
        self.val_set = FashionMNIST(root='./data', train=False,download=True, transform=self.val_transform)
      else:
        self.train_set = CIFAR10(root='./data', train=True,download=True, transform=self.train_transform)
        self.val_set = CIFAR10(root='./data', train=False,download=True, transform=self.val_transform)

    # This functions sets up the data loaders
    def setup(self):
        self.train_data_loader = torch.utils.data.DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)
        self.val_data_loader = torch.utils.data.DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False)

    # This is simply a getter function for the training data loader
    def train_dataloader(self):
        return self.train_data_loader

    # This is simply a getter function for the validation data loader
    def val_dataloader(self):
        return self.val_data_loader

# Setup the Model Class
Here we will define the model, its forward pass and its behaviour during each training/validation iteration

In [4]:
class DLModel(torch.nn.Module):
     def __init__(self, num_classes, pretrained=True, num_unfreeze_layers=0):
        super().__init__()
        # If you want to use the imagenet pretrained weights
        if pretrained:
            self.backbone = resnet18(weights=ResNet18_Weights.DEFAULT)
            # We freeze the entire model
            self.backbone.requires_grad_(False)
            # If you want to unfreeze some of the layers, then
            if num_unfreeze_layers > 0:
                # First find number of layers
                num_layers = 0
                for name, module in self.named_modules():
                    if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear) or isinstance(module, torch.nn.BatchNorm2d):
                        num_layers+=1
                # Following which unfreeze the last set of layers
                start_unfreezing_counter, counter = num_layers - num_unfreeze_layers, 0
                for name, module in self.named_modules():
                    if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear) or isinstance(module, torch.nn.BatchNorm2d):
                        counter+=1
                    if counter >= start_unfreezing_counter:
                        module.requires_grad_(True)
        # Otherwise just initialize the network from scratch
        else:
            self.backbone = resnet18(weights=None)
            self.backbone.requires_grad_(True)
        # The resnet model comes with a 1000 neuron final layer for the imagenet dataset
        self.backbone.fc = torch.nn.Sequential(torch.nn.Linear(512, num_classes)
        )
        self.backbone.fc.requires_grad_(True)
        # for name, module in self.named_modules():
        #     print(name, all(param.requires_grad for param in module.parameters()))
        # Define the objective function
        self.criterion = torch.nn.CrossEntropyLoss()
        # Define the metrics
        self.train_acc1, self.val_acc1 = Accuracy(task="multiclass", num_classes=num_classes), Accuracy(task="multiclass", num_classes=num_classes)
        self.train_acc5, self.val_acc5 = Accuracy(task="multiclass", num_classes=num_classes, top_k=5), Accuracy(task="multiclass", num_classes=num_classes, top_k=5)

    # This function sets up the optimizer and scheduler that we will use
     def configure_optimizers(self, lr, momentum, max_epochs):
        self.optimizer = torch.optim.SGD(self.parameters(), lr=lr, momentum=momentum)
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=max_epochs)

    # This defines the behaviour of our model during the forward pass
    # Based on the defined behaviour, PyTorch sets up the backward pass
     def forward(self, x):
        out = self.backbone(x)
        return out

    # This function describes model behaviour per iteration during training
     def training_step(self, x, y):
        self.optimizer.zero_grad()
        preds = self.forward(x)
        self.train_acc1.update(preds, y)
        self.train_acc5.update(preds, y)
        loss = self.criterion(preds, y)
        loss.backward()
        self.optimizer.step()
        return loss.item()

    # Once the epoch is complete, we can call this function for inspecting the model's performance
     def on_training_epoch_end(self, loss, epoch):
        acc1, acc5 = self.train_acc1.compute().item(), self.train_acc5.compute().item()
        print(f"Epoch No: {epoch+1}\nTraining Loss: {loss}\n Training Accuracy: {acc1} (Top-1)\t  {acc5} (Top-5)")
        return acc1, acc5

    # This function describes model behaviour per iteration during validation
     def validation_step(self, x, y):
        preds = self.forward(x)
        self.val_acc1.update(preds, y)
        self.val_acc5.update(preds, y)
        loss = self.criterion(preds, y)
        return loss.item()

    # Once the validation iterations are complete, we can call this function for inspecting the model's performance
     def on_validation_epoch_end(self, loss, epoch):
        acc1, acc5 = self.val_acc1.compute().item(), self.val_acc5.compute().item()
        print(f"Validation Loss: {loss}\nValidation Accuracy: {acc1} (Top-1)\t  {acc5} (Top-5)")
        return acc1, acc5

    # This function resets the metrics so that new results can be calculated for the next epoch
     def reset_metrics(self):
        self.train_acc1.reset(), self.train_acc5.reset()
        self.val_acc1.reset(), self.val_acc5.reset()


# Putting it all together
We first define some static and global variables

In [5]:
# First define some static variables
num_classes = 100
num_epochs = 10
batch_size = 128
# Fine-tuning and training from scratch require different sets of learning rates
lr_finetune, lr_scratch = 0.001, 0.1
momentum = 0.9
device = torch.device("cuda")

# Shallow Fine-tuning
We first take a look at shallow fine-tuning (training only the final layer and keeping the remaining model frozen)

## CIFAR10 Dataset

In [6]:
# Define the data
data_module = DataModule(batch_size=batch_size)
data_module.prepare_data()
data_module.setup()
train_loader, val_loader = data_module.train_dataloader(),data_module.val_dataloader()

# This variable will be used to save the per-epoch validation accuracy
shallow_finetuning_cifar10_val_acc = list()
# This variable will be used to save the per-epoch training loss
shallow_finetuning_cifar10_train_loss = list()

# Define the model
model = DLModel(num_classes=num_classes, pretrained=True,num_unfreeze_layers=0).to(device)
model.configure_optimizers(lr=lr_finetune, momentum=momentum,max_epochs=num_epochs)

# Start the training loop
for epoch in range(num_epochs):
  # This is the training cycle
  model.train()
  avg_loss = 0
  # Iterate over each batch and update the model
  for x, y in tqdm(train_loader, total=len(train_loader)):
    x, y = x.to(device), y.to(device)
    avg_loss+= model.training_step(x, y)
  avg_loss/=len(train_loader)
  shallow_finetuning_cifar10_train_loss.append(avg_loss)
  acc1, acc5 = model.on_training_epoch_end(avg_loss, epoch)

  # This is the validation cycle
  model.eval()
  avg_loss = 0
  # Iterate over each batch and get the predictions
  for x, y in tqdm(val_loader, total=len(val_loader)):
    x, y = x.to(device), y.to(device)
    avg_loss+= model.validation_step(x, y)
  avg_loss/=len(val_loader)
  acc1, acc5 = model.on_validation_epoch_end(avg_loss, epoch)
  shallow_finetuning_cifar10_val_acc.append(acc1)
  # Finally reset the metrics before going on to the next epoch
  model.reset_metrics()


100%|██████████| 170498071/170498071 [00:10<00:00, 15694254.10it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 51.0MB/s]


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

##  FashionMNIST Dataset

In [ ]:
# Define the data
data_module = DataModule(batch_size=batch_size, fmnist=True)
data_module.prepare_data()
data_module.setup()
train_loader, val_loader = data_module.train_dataloader(),data_module.val_dataloader()
# This variable will be used to save the per-epoch validation accuracy
shallow_finetuning_mnist_val_acc = list()
# This variable will be used to save the per-epoch training loss
shallow_finetuning_mnist_train_loss = list()

# Define the model
model = DLModel(num_classes=num_classes, pretrained=True,num_unfreeze_layers=0).to(device)
model.configure_optimizers(lr=lr_finetune, momentum=momentum,max_epochs=num_epochs)

# Start the training loop
for epoch in range(num_epochs):
  # This is the training cycle
  model.train()
  avg_loss = 0
  # Iterate over each batch and update the model
  for x, y in tqdm(train_loader, total=len(train_loader)):
    x, y = x.to(device), y.to(device)
    avg_loss+= model.training_step(x, y)
  avg_loss/=len(train_loader)
  shallow_finetuning_mnist_train_loss.append(avg_loss)
  acc1, acc5 = model.on_training_epoch_end(avg_loss, epoch)

  # This is the validation cycle
  model.eval()
  avg_loss = 0
  # Iterate over each batch and get the predictions
  for x, y in tqdm(val_loader, total=len(val_loader)):
    x, y = x.to(device), y.to(device)
    avg_loss+= model.validation_step(x, y)
  avg_loss/=len(val_loader)
  acc1, acc5 = model.on_validation_epoch_end(avg_loss, epoch)
  shallow_finetuning_mnist_val_acc.append(acc1)
  # Finally reset the metrics before going on to the next epoch
  model.reset_metrics()

# Deep Tunning
We now take a look at deep fine-tuning (training some the final layers while keeping the remaining layers frozen)

## CIFAR10 Dataset

In [ ]:
# Define the data
data_module = DataModule(batch_size=batch_size)
data_module.prepare_data()
data_module.setup()
train_loader, val_loader = data_module.train_dataloader(),data_module.val_dataloader()

# This variable will be used to save the per-epoch validation accuracy
deep_finetuning_cifar10_val_acc = list()
# This variable will be used to save the per-epoch training loss
deep_finetuning_cifar10_train_loss = list()

# Define the model
model = DLModel(num_classes=num_classes, pretrained=True,num_unfreeze_layers=31).to(device)
model.configure_optimizers(lr=lr_finetune, momentum=momentum,max_epochs=num_epochs)

# Start the training loop
for epoch in range(num_epochs):
  # This is the training cycle
  model.train()
  avg_loss = 0
  # Iterate over each batch and update the model
  for x, y in tqdm(train_loader, total=len(train_loader)):
    x, y = x.to(device), y.to(device)
    avg_loss+= model.training_step(x, y)
  avg_loss/=len(train_loader)
  deep_finetuning_cifar10_train_loss.append(avg_loss)
  acc1, acc5 = model.on_training_epoch_end(avg_loss, epoch)

  # This is the validation cycle
  model.eval()
  avg_loss = 0
  # Iterate over each batch and get the predictions
  for x, y in tqdm(val_loader, total=len(val_loader)):
    x, y = x.to(device), y.to(device)
    avg_loss+= model.validation_step(x, y)
  avg_loss/=len(val_loader)
  acc1, acc5 = model.on_validation_epoch_end(avg_loss, epoch)
  deep_finetuning_cifar10_val_acc.append(acc1)
  # Finally reset the metrics before going on to the next epoch
  model.reset_metrics()


## FashionMNIST Dataset

In [ ]:
# Define the data
data_module = DataModule(batch_size=batch_size, fmnist=True)
data_module.prepare_data()
data_module.setup()
train_loader, val_loader = data_module.train_dataloader(),data_module.val_dataloader()
# This variable will be used to save the per-epoch validation accuracy
deep_finetuning_mnist_val_acc = list()
# This variable will be used to save the per-epoch training loss
deep_finetuning_mnist_train_loss = list()

# Define the model
model = DLModel(num_classes=num_classes, pretrained=True,num_unfreeze_layers=31).to(device)
model.configure_optimizers(lr=lr_finetune, momentum=momentum,max_epochs=num_epochs)

# Start the training loop
for epoch in range(num_epochs):
  # This is the training cycle
  model.train()
  avg_loss = 0
  # Iterate over each batch and update the model
  for x, y in tqdm(train_loader, total=len(train_loader)):
    x, y = x.to(device), y.to(device)
    avg_loss+= model.training_step(x, y)
  avg_loss/=len(train_loader)
  deep_finetuning_mnist_train_loss.append(avg_loss)
  acc1, acc5 = model.on_training_epoch_end(avg_loss, epoch)

  # This is the validation cycle
  model.eval()
  avg_loss = 0
  # Iterate over each batch and get the predictions
  for x, y in tqdm(val_loader, total=len(val_loader)):
    x, y = x.to(device), y.to(device)
    avg_loss+= model.validation_step(x, y)
  avg_loss/=len(val_loader)
  acc1, acc5 = model.on_validation_epoch_end(avg_loss, epoch)
  deep_finetuning_mnist_val_acc.append(acc1)
  # Finally reset the metrics before going on to the next epoch
  model.reset_metrics()

# Training from Scratch
Finally, we train the ResNet18 model from scratch.

## CIFAR10 Dataset

In [ ]:
# Define the data
data_module = DataModule(batch_size=batch_size)
data_module.prepare_data()
data_module.setup()
train_loader, val_loader = data_module.train_dataloader(),data_module.val_dataloader()

# This variable will be used to save the per-epoch validation accuracy
scratch_cifar10_val_acc = list()
# This variable will be used to save the per-epoch training loss
scratch_cifar10_train_loss = list()

# Define the model
model = DLModel(num_classes=num_classes, pretrained=False,num_unfreeze_layers=31).to(device)
model.configure_optimizers(lr=lr_finetune, momentum=momentum,max_epochs=num_epochs)

# Start the training loop
for epoch in range(num_epochs):
  # This is the training cycle
  model.train()
  avg_loss = 0
  # Iterate over each batch and update the model
  for x, y in tqdm(train_loader, total=len(train_loader)):
    x, y = x.to(device), y.to(device)
    avg_loss+= model.training_step(x, y)
  avg_loss/=len(train_loader)
  scratch_cifar10_train_loss.append(avg_loss)
  acc1, acc5 = model.on_training_epoch_end(avg_loss, epoch)

  # This is the validation cycle
  model.eval()
  avg_loss = 0
  # Iterate over each batch and get the predictions
  for x, y in tqdm(val_loader, total=len(val_loader)):
    x, y = x.to(device), y.to(device)
    avg_loss+= model.validation_step(x, y)
  avg_loss/=len(val_loader)
  acc1, acc5 = model.on_validation_epoch_end(avg_loss, epoch)
  scratch_cifar10_val_acc.append(acc1)
  # Finally reset the metrics before going on to the next epoch
  model.reset_metrics()


## FashionMNIST Dataset

In [ ]:
# Define the data
data_module = DataModule(batch_size=batch_size, fmnist=True)
data_module.prepare_data()
data_module.setup()
train_loader, val_loader = data_module.train_dataloader(),data_module.val_dataloader()
# This variable will be used to save the per-epoch validation accuracy
scratch_mnist_val_acc = list()
# This variable will be used to save the per-epoch training loss
scratch_mnist_train_loss = list()

# Define the model
model = DLModel(num_classes=num_classes, pretrained=False,num_unfreeze_layers=31).to(device)
model.configure_optimizers(lr=lr_finetune, momentum=momentum,max_epochs=num_epochs)

# Start the training loop
for epoch in range(num_epochs):
  # This is the training cycle
  model.train()
  avg_loss = 0
  # Iterate over each batch and update the model
  for x, y in tqdm(train_loader, total=len(train_loader)):
    x, y = x.to(device), y.to(device)
    avg_loss+= model.training_step(x, y)
  avg_loss/=len(train_loader)
  scratch_mnist_train_loss.append(avg_loss)
  acc1, acc5 = model.on_training_epoch_end(avg_loss, epoch)

  # This is the validation cycle
  model.eval()
  avg_loss = 0
  # Iterate over each batch and get the predictions
  for x, y in tqdm(val_loader, total=len(val_loader)):
    x, y = x.to(device), y.to(device)
    avg_loss+= model.validation_step(x, y)
  avg_loss/=len(val_loader)
  acc1, acc5 = model.on_validation_epoch_end(avg_loss, epoch)
  scratch_mnist_val_acc.append(acc1)
  # Finally reset the metrics before going on to the next epoch
  model.reset_metrics()

# Comparisons
Now that we have evaluated the models under three different forms of training, let us compare them.

## CIFAR10 Dataset

In [ ]:
# Comparing the Training loss per epoch
sns.lineplot(x=np.arange(len(shallow_finetuning_cifar10_train_loss)), y=shallow_finetuning_cifar10_train_loss, label="Shallow Fine-tuning")
sns.lineplot(x=np.arange(len(deep_finetuning_cifar10_train_loss)), y=deep_finetuning_cifar10_train_loss, label="Deep Fine-tuning")
sns.lineplot(x=np.arange(len(scratch_cifar10_train_loss)), y=scratch_cifar10_train_loss, label="Training from scratch")
plt.xlabel("Epochs")
plt.ylabel("Training Loss")
sns.despine()
plt.title("Training Loss Comparison")
plt.show()
plt.close()

# Comparing the validation Accuracies per epoch
sns.lineplot(x=np.arange(len(shallow_finetuning_cifar10_val_acc)), y=shallow_finetuning_cifar10_val_acc, label="Shallow Fine-tuning")
sns.lineplot(x=np.arange(len(deep_finetuning_cifar10_val_acc)), y=deep_finetuning_cifar10_val_acc, label="Deep Fine-tuning")
sns.lineplot(x=np.arange(len(scratch_cifar10_val_acc)), y=scratch_cifar10_val_acc, label="Training from scratch")
plt.xlabel("Epochs")
plt.ylabel("Validation Accuracy")
sns.despine()
plt.title("Validation Accuracy Comparison")
plt.show()

## FashionMNIST Dataset

In [ ]:
# Comparing the Training loss per epoch
sns.lineplot(x=np.arange(len(shallow_finetuning_mnist_train_loss)), y=shallow_finetuning_mnist_train_loss, label="Shallow Fine-tuning")
sns.lineplot(x=np.arange(len(deep_finetuning_mnist_train_loss)), y=deep_finetuning_mnist_train_loss, label="Deep Fine-tuning")
sns.lineplot(x=np.arange(len(scratch_mnist_train_loss)), y=scratch_mnist_train_loss, label="Training from scratch")
plt.xlabel("Epochs")
plt.ylabel("Training Loss")
sns.despine()
plt.title("Training Loss Comparison")
plt.show()
plt.close()

# Comparing the validation Accuracies per epoch
sns.lineplot(x=np.arange(len(shallow_finetuning_mnist_val_acc)), y=shallow_finetuning_mnist_val_acc, label="Shallow Fine-tuning")
sns.lineplot(x=np.arange(len(deep_finetuning_mnist_val_acc)), y=deep_finetuning_mnist_val_acc, label="Deep Fine-tuning")
sns.lineplot(x=np.arange(len(scratch_mnist_val_acc)), y=scratch_mnist_val_acc, label="Training from scratch")
plt.xlabel("Epochs")
plt.ylabel("Validation Accuracy")
sns.despine()
plt.title("Validation Accuracy Comparison")
plt.show()